# Dependencies

In [ ]:
import IPython.display as display
!pip uninstall -y requests

!pip install -U urllib3 requests
display.clear_output()

In [ ]:
!pip uninstall -y torch torchvision
!pip install torch==2.0.0 torchvision==0.15.1

In [ ]:
!pip install -U openmim
!pip install "mmengine>=0.7.1,<1.0.0" \
 "mmcv>=2.0.0rc4,<2.1.0" \
 -f https://download.openmmlab.com/mmcv/dist/cu117/torch2.0.0/index.html \
--trusted-host download.openmmlab.com

In [ ]:
# Install mmdetection
!rm -rf mmdetection
!git clone https://github.com/open-mmlab/mmdetection.git
%cd mmdetection

%pip install -e .

# Import Dataset

In [ ]:
#copying the dataset to mmyolo directory

import shutil
import os

# Source directory to copy
src_dir = '/kaggle/input/loserspcb-v2/combined_trialv4_updated'

# Destination directory where the source directory will be copied
dst_dir = '/kaggle/working/mmdetection/datasets'

# Remove destination directory if it exists
if os.path.exists(dst_dir):
    shutil.rmtree(dst_dir)

# Copy the entire directory tree
shutil.copytree(src_dir, dst_dir)

# Config modification 

In [ ]:
config_coco = """


dataset_type = 'CocoDataset'
data_root = './datasets/'

# Example to use different file client
# Method 1: simply set the data root and let the file I/O module
# automatically infer from prefix (not support LMDB and Memcache yet)

# data_root = 's3://openmmlab/datasets/detection/coco/'

train_batch_size_per_gpu = 1
val_batch_size_per_gpu = 1
train_num_workers = 1 
val_num_workers = 1 
metainfo = {
    'classes': ('MP','OC','SC','SP','SPC')
    }

# Method 2: Use `backend_args`, `file_client_args` in versions before 3.0.0rc6
# backend_args = dict(
#     backend='petrel',
#     path_mapping=dict({
#         './data/': 's3://openmmlab/datasets/detection/',
#         'data/': 's3://openmmlab/datasets/detection/'
#     }))
backend_args = None

train_pipeline = [
    dict(type='LoadImageFromFile', backend_args=backend_args),
    dict(type='LoadAnnotations', with_bbox=True),
    dict(type='Resize', scale=(960,960), keep_ratio=True),
    dict(type='PackDetInputs')
]
test_pipeline = [
    dict(type='LoadImageFromFile', backend_args=backend_args),
    dict(type='Resize', scale=(960,960), keep_ratio=True),
    # If you don't have a gt annotation, delete the pipeline
    dict(type='LoadAnnotations', with_bbox=True),
    dict(
        type='PackDetInputs',
        meta_keys=('img_id', 'img_path', 'ori_shape', 'img_shape',
                   'scale_factor'))
]
train_dataloader = dict(
    batch_size=train_batch_size_per_gpu,
    num_workers=train_num_workers,
    persistent_workers=True,
    sampler=dict(type='DefaultSampler', shuffle=True),
    batch_sampler=dict(type='AspectRatioBatchSampler'),
    dataset=dict(
        type=dataset_type,
        data_root=data_root,
        metainfo=metainfo,
        ann_file='train.json',
        data_prefix=dict(img='train/'),
        filter_cfg=dict(filter_empty_gt=True, min_size=32),
        pipeline=train_pipeline,
        backend_args=backend_args))
val_dataloader = dict(
    batch_size=val_batch_size_per_gpu,
    num_workers=val_num_workers,
    persistent_workers=True,
    drop_last=False,
    sampler=dict(type='DefaultSampler', shuffle=False),
    dataset=dict(
        type=dataset_type,
        data_root=data_root,
        metainfo=metainfo,
        ann_file='val.json',
        data_prefix=dict(img='val/'),
        test_mode=True,
        pipeline=test_pipeline,
        backend_args=backend_args))
test_dataloader = val_dataloader

val_evaluator = dict(
    type='CocoMetric',
    ann_file=data_root + 'val.json',
    metric='bbox',
    classwise=True,
    format_only=False,
    backend_args=backend_args)
test_evaluator = val_evaluator



"""
with open('./configs/_base_/datasets/coco_detection.py', 'w') as f:
    f.write(config_coco)

In [ ]:
#config for augmentation
#/kaggle/working/mmdetection/mmdet/configs/common/ms_3x_coco.py
#config for model
#/kaggle/working/mmdetection/projects/CO-DETR/configs/codino/co_dino_5scale_r50_lsj_8xb2_1x_coco.py

In [ ]:
#/kaggle/working/mmdetection/configs/_base_/datasets/coco_detection.py

In [ ]:
#for visualising the config
from mmengine import Config
import json
cfg = Config.fromfile('/kaggle/working/mmdetection/configs/_base_/datasets/coco_detection.py')
formatted_cfg = json.dumps(cfg._cfg_dict, indent=4)

print(formatted_cfg)

In [ ]:
config_pcb_defect = """

_base_ = [
    '../_base_/default_runtime.py', '../_base_/schedules/schedule_2x.py',
    '../_base_/datasets/coco_detection.py'
]

custom_imports = dict(
    imports=['projects.CO-DETR.codetr'], allow_failed_imports=False)

# model settings
num_dec_layer = 6
loss_lambda = 2.0
num_classes = 5


model = dict(
    type='CoDETR',
    # If using the lsj augmentation,
    # it is recommended to set it to True.
    use_lsj=False, #Not using any lsj based augmentation 
    # detr: 52.1
    # one-stage: 49.4
    # two-stage: 47.9
    eval_module='detr',  # in ['detr', 'one-stage', 'two-stage']
    data_preprocessor=dict(
        type='DetDataPreprocessor',
        mean=[123.675, 116.28, 103.53],
        std=[58.395, 57.12, 57.375],
        bgr_to_rgb=True,
        pad_mask=False,
        batch_augments=None),
    backbone=dict(
        type='ResNet',
        depth=50,
        num_stages=4,
        out_indices=(0, 1, 2, 3),
        frozen_stages=1,
        norm_cfg=dict(type='BN', requires_grad=False),
        norm_eval=True,
        style='pytorch',
        init_cfg=dict(type='Pretrained', checkpoint='torchvision://resnet50')),
    neck=dict(
        type='ChannelMapper',
        in_channels=[256, 512, 1024, 2048],
        kernel_size=1,
        out_channels=256,
        act_cfg=None,
        norm_cfg=dict(type='GN', num_groups=32),
        num_outs=5),
    query_head=dict(
        type='CoDINOHead',
        num_query=900,
        num_classes=num_classes,
        in_channels=2048,
        as_two_stage=True,
        dn_cfg=dict(
            label_noise_scale=0.5,
            box_noise_scale=1.0,
            group_cfg=dict(dynamic=True, num_groups=None, num_dn_queries=100)),
        transformer=dict(
            type='CoDinoTransformer',
            with_coord_feat=False,
            num_co_heads=2,  # ATSS Aux Head + Faster RCNN Aux Head
            num_feature_levels=5,
            encoder=dict(
                type='DetrTransformerEncoder',
                num_layers=6,
                # number of layers that use checkpoint.
                # The maximum value for the setting is num_layers.
                # FairScale must be installed for it to work.
                with_cp=4,
                transformerlayers=dict(
                    type='BaseTransformerLayer',
                    attn_cfgs=dict(
                        type='MultiScaleDeformableAttention',
                        embed_dims=256,
                        num_levels=5,
                        dropout=0.0),
                    feedforward_channels=2048,
                    ffn_dropout=0.0,
                    operation_order=('self_attn', 'norm', 'ffn', 'norm'))),
            decoder=dict(
                type='DinoTransformerDecoder',
                num_layers=6,
                return_intermediate=True,
                transformerlayers=dict(
                    type='DetrTransformerDecoderLayer',
                    attn_cfgs=[
                        dict(
                            type='MultiheadAttention',
                            embed_dims=256,
                            num_heads=8,
                            dropout=0.0),
                        dict(
                            type='MultiScaleDeformableAttention',
                            embed_dims=256,
                            num_levels=5,
                            dropout=0.0),
                    ],
                    feedforward_channels=2048,
                    ffn_dropout=0.0,
                    operation_order=('self_attn', 'norm', 'cross_attn', 'norm',
                                     'ffn', 'norm')))),
        positional_encoding=dict(
            type='SinePositionalEncoding',
            num_feats=128,
            temperature=20,
            normalize=True),
        loss_cls=dict(  # Different from the DINO
            type='QualityFocalLoss',
            use_sigmoid=True,
            beta=2.0,
            loss_weight=1.0),
        loss_bbox=dict(type='L1Loss', loss_weight=5.0),
        loss_iou=dict(type='GIoULoss', loss_weight=2.0)),
    rpn_head=dict(
        type='RPNHead',
        in_channels=256,
        feat_channels=256,
        anchor_generator=dict(
            type='AnchorGenerator',
            octave_base_scale=4,
            scales_per_octave=3,
            ratios=[0.5, 1.0, 2.0],
            strides=[4, 8, 16, 32, 64, 128]),
        bbox_coder=dict(
            type='DeltaXYWHBBoxCoder',
            target_means=[.0, .0, .0, .0],
            target_stds=[1.0, 1.0, 1.0, 1.0]),
        loss_cls=dict(
            type='CrossEntropyLoss',
            use_sigmoid=True,
            loss_weight=1.0 * num_dec_layer * loss_lambda),
        loss_bbox=dict(
            type='L1Loss', loss_weight=1.0 * num_dec_layer * loss_lambda)),
    roi_head=[
        dict(
            type='CoStandardRoIHead',
            bbox_roi_extractor=dict(
                type='SingleRoIExtractor',
                roi_layer=dict(
                    type='RoIAlign', output_size=7, sampling_ratio=0),
                out_channels=256,
                featmap_strides=[4, 8, 16, 32, 64],
                finest_scale=56),
            bbox_head=dict(
                type='Shared2FCBBoxHead',
                in_channels=256,
                fc_out_channels=1024,
                roi_feat_size=7,
                num_classes=num_classes,
                bbox_coder=dict(
                    type='DeltaXYWHBBoxCoder',
                    target_means=[0., 0., 0., 0.],
                    target_stds=[0.1, 0.1, 0.2, 0.2]),
                reg_class_agnostic=False,
                reg_decoded_bbox=True,
                loss_cls=dict(
                    type='CrossEntropyLoss',
                    use_sigmoid=False,
                    loss_weight=1.0 * num_dec_layer * loss_lambda),
                loss_bbox=dict(
                    type='GIoULoss',
                    loss_weight=10.0 * num_dec_layer * loss_lambda)))
    ],
    bbox_head=[
        dict(
            type='CoATSSHead',
            num_classes=num_classes,
            in_channels=256,
            stacked_convs=1,
            feat_channels=256,
            anchor_generator=dict(
                type='AnchorGenerator',
                ratios=[1.0],
                octave_base_scale=8,
                scales_per_octave=1,
                strides=[4, 8, 16, 32, 64, 128]),
            bbox_coder=dict(
                type='DeltaXYWHBBoxCoder',
                target_means=[.0, .0, .0, .0],
                target_stds=[0.1, 0.1, 0.2, 0.2]),
            loss_cls=dict(
                type='FocalLoss',
                use_sigmoid=True,
                gamma=2.0,
                alpha=0.25,
                loss_weight=1.0 * num_dec_layer * loss_lambda),
            loss_bbox=dict(
                type='GIoULoss',
                loss_weight=2.0 * num_dec_layer * loss_lambda),
            loss_centerness=dict(
                type='CrossEntropyLoss',
                use_sigmoid=True,
                loss_weight=1.0 * num_dec_layer * loss_lambda)),
    ],
    # model training and testing settings
    train_cfg=[
        dict(
            assigner=dict(
                type='HungarianAssigner',
                match_costs=[
                    dict(type='FocalLossCost', weight=2.0),
                    dict(type='BBoxL1Cost', weight=5.0, box_format='xywh'),
                    dict(type='IoUCost', iou_mode='giou', weight=2.0)
                ])),
        dict(
            rpn=dict(
                assigner=dict(
                    type='MaxIoUAssigner',
                    pos_iou_thr=0.7,
                    neg_iou_thr=0.3,
                    min_pos_iou=0.3,
                    match_low_quality=True,
                    ignore_iof_thr=-1),
                sampler=dict(
                    type='RandomSampler',
                    num=256,
                    pos_fraction=0.5,
                    neg_pos_ub=-1,
                    add_gt_as_proposals=False),
                allowed_border=-1,
                pos_weight=-1,
                debug=False),
            rpn_proposal=dict(
                nms_pre=4000,
                max_per_img=1000,
                nms=dict(type='nms', iou_threshold=0.7),
                min_bbox_size=0),
            rcnn=dict(
                assigner=dict(
                    type='MaxIoUAssigner',
                    pos_iou_thr=0.5,
                    neg_iou_thr=0.5,
                    min_pos_iou=0.5,
                    match_low_quality=False,
                    ignore_iof_thr=-1),
                sampler=dict(
                    type='RandomSampler',
                    num=512,
                    pos_fraction=0.25,
                    neg_pos_ub=-1,
                    add_gt_as_proposals=True),
                pos_weight=-1,
                debug=False)),
        dict(
            assigner=dict(type='ATSSAssigner', topk=9),
            allowed_border=-1,
            pos_weight=-1,
            debug=False)
    ],
    test_cfg=[
        # Deferent from the DINO, we use the NMS.
        dict(
            max_per_img=300,
            # NMS can improve the mAP by 0.2.
            nms=dict(type='soft_nms', iou_threshold=0.8)),
        dict(
            rpn=dict(
                nms_pre=1000,
                max_per_img=1000,
                nms=dict(type='nms', iou_threshold=0.7),
                min_bbox_size=0),
            rcnn=dict(
                score_thr=0.0,
                nms=dict(type='nms', iou_threshold=0.5),
                max_per_img=100)),
        dict(
            # atss bbox head:
            nms_pre=1000,
            min_bbox_size=0,
            score_thr=0.0,
            nms=dict(type='nms', iou_threshold=0.6),
            max_per_img=100),
        # soft-nms is also supported for rcnn testing
        # e.g., nms=dict(type='soft_nms', iou_threshold=0.5, min_score=0.05)
    ])

max_epochs = 12
# training schedule for 2x
train_cfg = dict(
    _delete_=True,
    type='EpochBasedTrainLoop',
    max_epochs=max_epochs,
    val_interval=1)
    

optim_wrapper = dict(
    _delete_=True,
    type='OptimWrapper',
    optimizer=dict(type='AdamW', lr=2e-4, weight_decay=0.0001),
    clip_grad=dict(max_norm=0.1, norm_type=2),
    paramwise_cfg=dict(custom_keys={'backbone': dict(lr_mult=0.1)}))

val_evaluator = dict(metric='bbox')
test_evaluator = val_evaluator

param_scheduler = [
    dict(
        type='MultiStepLR',
        begin=0,
        end=max_epochs,
        by_epoch=True,
        milestones=[11],
        gamma=0.1)
]

default_hooks = dict(
    checkpoint=dict(by_epoch=True, interval=1, max_keep_ckpts=1,save_best='auto'))
log_processor = dict(by_epoch=True)


# NOTE: `auto_scale_lr` is for automatically scaling LR,
# USER SHOULD NOT CHANGE ITS VALUES.
# base_batch_size = (8 GPUs) x (2 samples per GPU)
auto_scale_lr = dict(base_batch_size=2)

"""
with open('/kaggle/working/mmdetection/configs/detr/co_dino_5scale_r50_lsj_8xb2_1x_pcb_defect.py', 'w') as f:
    f.write(config_pcb_defect)

In [ ]:
#load_from = '/kaggle/input/weights/co_dino_5scale_r50_lsj_8xb2_1x_coco-69a72d67.pth'

In [ ]:
#for visualising the config
from mmengine import Config
import json
cfg = Config.fromfile('/kaggle/working/mmdetection/configs/detr/co_dino_5scale_r50_lsj_8xb2_1x_pcb_defect.py')
formatted_cfg = json.dumps(cfg._cfg_dict, indent=4)

print(formatted_cfg)

In [ ]:
# saving the config file
from mmengine import Config
import json

# Load the configuration from file
cfg = Config.fromfile('/kaggle/working/mmdetection/configs/detr/co_dino_5scale_r50_lsj_8xb2_1x_pcb_defect.py')

# Convert the configuration to a dictionary and then to a formatted JSON string
formatted_cfg = json.dumps(cfg._cfg_dict, indent=4)

# Define the output file path
output_file = '/kaggle/working/formatted_config.json'

# Save the formatted JSON string to a file
with open(output_file, 'w') as f:
    f.write(formatted_cfg)

print(f'Configuration saved as {output_file}')

# Visualizing the Dataset 

In [ ]:

# # Basically, as the browse_dataset code of mmdet does not work
# # i had to copy the browse_dataset code of mmyolo here, one line had to change
# # from mmyolo.registry import DATASETS, VISUALIZERS >>> from mmdet.registry import DATASETS, VISUALIZERS

# code ='''
# # Copyright (c) OpenMMLab. All rights reserved.
# import argparse
# import os.path as osp
# import sys
# from typing import Tuple

# import cv2
# import mmcv
# import numpy as np
# from mmdet.models.utils import mask2ndarray
# from mmdet.structures.bbox import BaseBoxes
# from mmengine.config import Config, DictAction
# from mmengine.dataset import Compose
# from mmengine.registry import init_default_scope
# from mmengine.utils import ProgressBar
# from mmengine.visualization import Visualizer

# from mmdet.registry import DATASETS, VISUALIZERS


# # TODO: Support for printing the change in key of results
# # TODO: Some bug. If you meet some bug, please use the original
# def parse_args():
#     parser = argparse.ArgumentParser(description='Browse a dataset')
#     parser.add_argument('config', help='train config file path')
#     parser.add_argument(
#         '--phase',
#         '-p',
#         default='train',
#         type=str,
#         choices=['train', 'test', 'val'],
#         help='phase of dataset to visualize, accept "train" "test" and "val".'
#         ' Defaults to "train".')
#     parser.add_argument(
#         '--mode',
#         '-m',
#         default='transformed',
#         type=str,
#         choices=['original', 'transformed', 'pipeline'],
#         help='display mode; display original pictures or '
#         'transformed pictures or comparison pictures. "original" '
#         'means show images load from disk; "transformed" means '
#         'to show images after transformed; "pipeline" means show all '
#         'the intermediate images. Defaults to "transformed".')
#     parser.add_argument(
#         '--out-dir',
#         default='output',
#         type=str,
#         help='If there is no display interface, you can save it.')
#     parser.add_argument('--not-show', default=False, action='store_true')
#     parser.add_argument(
#         '--show-number',
#         '-n',
#         type=int,
#         default=sys.maxsize,
#         help='number of images selected to visualize, '
#         'must bigger than 0. if the number is bigger than length '
#         'of dataset, show all the images in dataset; '
#         'default "sys.maxsize", show all images in dataset')
#     parser.add_argument(
#         '--show-interval',
#         '-i',
#         type=float,
#         default=3,
#         help='the interval of show (s)')
#     parser.add_argument(
#         '--cfg-options',
#         nargs='+',
#         action=DictAction,
#         help='override some settings in the used config, the key-value pair '
#         'in xxx=yyy format will be merged into config file. If the value to '
#         'be overwritten is a list, it should be like key="[a,b]" or key=a,b '
#         'It also allows nested list/tuple values, e.g. key="[(a,b),(c,d)]" '
#         'Note that the quotation marks are necessary and that no white space '
#         'is allowed.')
#     args = parser.parse_args()
#     return args


# def _get_adaptive_scale(img_shape: Tuple[int, int],
#                         min_scale: float = 0.3,
#                         max_scale: float = 3.0) -> float:
#     """Get adaptive scale according to image shape.

#     The target scale depends on the the short edge length of the image. If the
#     short edge length equals 224, the output is 1.0. And output linear
#     scales according the short edge length. You can also specify the minimum
#     scale and the maximum scale to limit the linear scale.

#     Args:
#         img_shape (Tuple[int, int]): The shape of the canvas image.
#         min_scale (int): The minimum scale. Defaults to 0.3.
#         max_scale (int): The maximum scale. Defaults to 3.0.
#     Returns:
#         int: The adaptive scale.
#     """
#     short_edge_length = min(img_shape)
#     scale = short_edge_length / 224.
#     return min(max(scale, min_scale), max_scale)


# def make_grid(imgs, names):
#     """Concat list of pictures into a single big picture, align height here."""
#     visualizer = Visualizer.get_current_instance()
#     ori_shapes = [img.shape[:2] for img in imgs]
#     max_height = int(max(img.shape[0] for img in imgs) * 1.1)
#     min_width = min(img.shape[1] for img in imgs)
#     horizontal_gap = min_width // 10
#     img_scale = _get_adaptive_scale((max_height, min_width))

#     texts = []
#     text_positions = []
#     start_x = 0
#     for i, img in enumerate(imgs):
#         pad_height = (max_height - img.shape[0]) // 2
#         pad_width = horizontal_gap // 2
#         # make border
#         imgs[i] = cv2.copyMakeBorder(
#             img,
#             pad_height,
#             max_height - img.shape[0] - pad_height + int(img_scale * 30 * 2),
#             pad_width,
#             pad_width,
#             cv2.BORDER_CONSTANT,
#             value=(255, 255, 255))
#         texts.append(f'{"execution: "}{i}\\n{names[i]}\\n{ori_shapes[i]}')

#         text_positions.append(
#             [start_x + img.shape[1] // 2 + pad_width, max_height])
#         start_x += img.shape[1] + horizontal_gap

#     display_img = np.concatenate(imgs, axis=1)
#     visualizer.set_image(display_img)
#     img_scale = _get_adaptive_scale(display_img.shape[:2])
#     visualizer.draw_texts(
#         texts,
#         positions=np.array(text_positions),
#         font_sizes=img_scale * 7,
#         colors='black',
#         horizontal_alignments='center',
#         font_families='monospace')
#     return visualizer.get_image()


# def swap_pipeline_position(dataset_cfg):
#     load_ann_tfm_name = 'LoadAnnotations'
#     pipeline = dataset_cfg.get('pipeline')
#     if (pipeline is None):
#         return dataset_cfg
#     all_transform_types = [tfm['type'] for tfm in pipeline]
#     if load_ann_tfm_name in all_transform_types:
#         load_ann_tfm_index = all_transform_types.index(load_ann_tfm_name)
#         load_ann_tfm = pipeline.pop(load_ann_tfm_index)
#         pipeline.insert(1, load_ann_tfm)


# class InspectCompose(Compose):
#     """Compose multiple transforms sequentially.

#     And record "img" field of all results in one list.
#     """

#     def __init__(self, transforms, intermediate_imgs):
#         super().__init__(transforms=transforms)
#         self.intermediate_imgs = intermediate_imgs

#     def __call__(self, data):
#         if 'img' in data:
#             self.intermediate_imgs.append({
#                 'name': 'original',
#                 'img': data['img'].copy()
#             })
#         self.ptransforms = [
#             self.transforms[i] for i in range(len(self.transforms) - 1)
#         ]
#         for t in self.ptransforms:
#             data = t(data)
#             # Keep the same meta_keys in the PackDetInputs
#             self.transforms[-1].meta_keys = [key for key in data]
#             data_sample = self.transforms[-1](data)
#             if data is None:
#                 return None
#             if 'img' in data:
#                 self.intermediate_imgs.append({
#                     'name':
#                     t.__class__.__name__,
#                     'dataset_sample':
#                     data_sample['data_samples']
#                 })
#         return data


# def main():
#     args = parse_args()
#     cfg = Config.fromfile(args.config)
#     if args.cfg_options is not None:
#         cfg.merge_from_dict(args.cfg_options)

#     init_default_scope(cfg.get('default_scope', 'mmyolo'))

#     dataset_cfg = cfg.get(args.phase + '_dataloader').get('dataset')
#     if (args.phase in ['test', 'val']):
#         swap_pipeline_position(dataset_cfg)
#     dataset = DATASETS.build(dataset_cfg)
#     visualizer = VISUALIZERS.build(cfg.visualizer)
#     visualizer.dataset_meta = dataset.metainfo

#     intermediate_imgs = []

#     if not hasattr(dataset, 'pipeline'):
#         # for dataset_wrapper
#         dataset = dataset.dataset

#     # TODO: The dataset wrapper occasion is not considered here
#     dataset.pipeline = InspectCompose(dataset.pipeline.transforms,
#                                       intermediate_imgs)

#     # init visualization image number
#     assert args.show_number > 0
#     display_number = min(args.show_number, len(dataset))

#     progress_bar = ProgressBar(display_number)
#     for i, item in zip(range(display_number), dataset):
#         image_i = []
#         result_i = [result['dataset_sample'] for result in intermediate_imgs]
#         for k, datasample in enumerate(result_i):
#             image = datasample.img
#             gt_instances = datasample.gt_instances
#             image = image[..., [2, 1, 0]]  # bgr to rgb
#             gt_bboxes = gt_instances.get('bboxes', None)
#             if gt_bboxes is not None and isinstance(gt_bboxes, BaseBoxes):
#                 gt_instances.bboxes = gt_bboxes.tensor
#             gt_masks = gt_instances.get('masks', None)
#             if gt_masks is not None:
#                 masks = mask2ndarray(gt_masks)
#                 gt_instances.masks = masks.astype(bool)
#                 datasample.gt_instances = gt_instances
#             # get filename from dataset or just use index as filename
#             visualizer.add_datasample(
#                 'result',
#                 image,
#                 datasample,
#                 draw_pred=False,
#                 draw_gt=True,
#                 show=False)
#             image_show = visualizer.get_image()
#             image_i.append(image_show)

#         if args.mode == 'original':
#             image = image_i[0]
#         elif args.mode == 'transformed':
#             image = image_i[-1]
#         else:
#             image = make_grid([result for result in image_i],
#                               [result['name'] for result in intermediate_imgs])

#         if hasattr(datasample, 'img_path'):
#             filename = osp.basename(datasample.img_path)
#         else:
#             # some dataset have not image path
#             filename = f'{i}.jpg'
#         out_file = osp.join(args.out_dir,
#                             filename) if args.out_dir is not None else None

#         if out_file is not None:
#             mmcv.imwrite(image[..., ::-1], out_file)

#         if not args.not_show:
#             visualizer.show(
#                 image, win_name=filename, wait_time=args.show_interval)

#         intermediate_imgs.clear()
#         progress_bar.update()


# if __name__ == '__main__':
#     main()
# '''

# with open('./tools/analysis_tools/browse_dataset_2.py', 'w') as f:
#     f.write(code)

In [ ]:
# !python tools/analysis_tools/browse_dataset_2.py configs/detr/co_dino_5scale_r50_lsj_8xb2_1x_pcb_defect.py --mode pipeline --out-dir dataset_check_6 --show-number 6 --show-interval 2

# Training

In [ ]:
#!python tools/train.py configs/detr/co_dino_5scale_r50_lsj_8xb2_1x_pcb_defect.py --work-dir co-detr_dspcbsd_12e/

In [ ]:
!bash ./tools/dist_train.sh configs/detr/co_dino_5scale_r50_lsj_8xb2_1x_pcb_defect.py 2 --work-dir co-detr_loserspcb_12e/ 

In [ ]:
# Just to look at the converted data
#Saving the weight files
import shutil

# Directory to be zipped
directory_to_zip = '/kaggle/working/mmdetection/co-detr_loserspcb_12e'

# Destination zip file path
zip_file_path = '/kaggle/working/co-detr_loserspcb_12e.zip'

# Create a zip file
shutil.make_archive(zip_file_path[:-4], 'zip', directory_to_zip)

# Testing 

In [ ]:
# !python tools/test.py \
# configs/rtmdet/rtmdet_m_8xb32-50e_pcb_defect.py \
# /kaggle/input/inference-2/best_coco_SH_precision_epoch_45.pth\
# --work-dir results/evaluate \
# --out results/results.pkl

# Flops

In [ ]:
# #I modified the get_flops scripts so that it works

# import shutil

# #Define the source file path and the destination file path
# source = '/kaggle/input/get-flops-update/get_flops_2.py'
# destination = '/kaggle/working/mmdetection/tools/analysis_tools/get_flops_2.py'

# #Copy the file
# shutil.copy(source, destination)

In [ ]:
# !python tools/analysis_tools/get_flops_2.py configs/rtmdet/rtmdet_m_8xb32-50e_pcb_defect.py

# Benchmark

In [ ]:
# !python tools/analysis_tools/benchmark.py configs/rtmdet/rtmdet_m_8xb32-50e_pcb_defect.py \
# --checkpoint /kaggle/input/inference-3/best_coco_bbox_mAP_epoch_45.pth \
# --task inference \
# --repeat-num 5 \
# --max-iter 100 \
# --log-interval 50 \
# --num-warmup 10 \
# --work-dir ./results

# Confusion Matrix

In [ ]:

# import shutil

# # Define the source file path and the destination file path
# source = '/kaggle/input/confusion-matrix-mod/confusion_matrix.py'
# destination = '/kaggle/working/mmdetection/tools/analysis_tools/confusion_matrix_4.py'

# # Copy the file
# shutil.copy(source, destination)

In [ ]:
# !python tools/analysis_tools/confusion_matrix_4.py \
# configs/rtmdet/rtmdet_m_8xb32-50e_pcb_defect.py  \
# results/results.pkl  \
# ./results \
# --show \
# --score-thr 0.5 \
# --tp-iou-thr 0.5

# Saving the results

In [ ]:
# import shutil

# # Directory to be zipped
# directory_to_zip = '/kaggle/working/mmdetection/results'

# # Destination zip file path
# zip_file_path = '/kaggle/working/rtmdet-m_dspcbsd_results.zip'

# # Create a zip file
# shutil.make_archive(zip_file_path[:-4], 'zip', directory_to_zip)

# Inference

In [ ]:
# from mmdet.apis import DetInferencer
# import glob

# # Choose to use a config
# config = 'configs/rtmdet/rtmdet_m_8xb32-50e_pcb_defect.py'
# # Setup a checkpoint file to load
# checkpoint = '/kaggle/input/inference-2/best_coco_SH_precision_epoch_45.pth'

# # Set the device to be used for evaluation
# device = 'cuda:0'

# # Initialize the DetInferencer
# inferencer = DetInferencer(config, checkpoint, device)

# # Use the detector to do inference
# img = '/kaggle/input/dspcbsd/DsPCBSD+/Data_COCO/val/0373736.jpg'

# result = inferencer(img, out_dir='./output',no_save_pred=False, pred_score_thr=0.5)

In [ ]:
# # Show the output image
# from PIL import Image
# Image.open('./output/vis/0373736.jpg')